# Initialize Environment

In [ ]:
import matplotlib, matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import numpy as np
from scipy.stats import norm
from scipy.io import loadmat
import glob
import pandas as pd
import datetime
import pandas_datareader.tsp as tsp
import pandas_datareader.data as pdr
import pandas_datareader.yahoo.daily as ydr
from pandas_datareader.yahoo.headers import DEFAULT_HEADERS
import requests_cache
use_yf = True
if use_yf:
    import yfinance as yf
    yf.pdr_override() # https://pypi.org/project/yfinance/

from copy import deepcopy
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Seed Random number generator for repeatability
np.random.seed(19690720)

plt.style.use('default')
#matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams.update({'font.size': 18})

# Load Data
## Data Caches

In [ ]:
session = requests_cache.CachedSession(cache_name='cache', 
                                       backend='sqlite', 
                                       expire_after=datetime.timedelta(hours=8))
session.headers = DEFAULT_HEADERS

## TSP
Load TSP data using the pandas_datareader utility. Allow for 8-hour persistent caching. 

In [ ]:
if False:
    tspreader = tsp.TSPReader(start=datetime.date.today() - datetime.timedelta(days=365*25),
                              session=session)
    full_tspdata = tspreader.read()

    # write the TSP data, last 90 values, in reversed order
    full_tspdata.iloc[:400].to_excel('tsp_shareprices.xlsx', sheet_name='TSP Data')

    #print(full_tspdata)
    tspdata = full_tspdata.drop(['L Income', 'L 2025', 'L 2030', 'L 2035', 'L 2040', 'L 2045', 'L 2050', 'L 2055', 'L 2060', 'L 2065'],axis=1).iloc[::-1]

## Vanguard

In [ ]:
write_xlsx = False
try:
    tickers
except NameError:    
    #tickers = [ 'VTSAX', 'VSGAX', 'VGSLX', 'VHYAX', 'VTMSX', 'VTCLX', 'VTMFX', 'VBTLX', 'VWAHX', 'VGSTX', 'VIHAX', 'VWIGX' ]
    tickers = [ 'VINIX', 'VSGIX', 'VSIIX', 'VTSNX', 'VBTIX', 'VGSNX', 'VTSAX' ]
    write_xlsx = True
    
# old API - requires yfinance & yf.pdr_override()    
if use_yf:
    data = pdr.get_data_yahoo(
            # tickers list or string as well
            tickers = tickers,

            # use "period" instead of start/end
            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            # (optional, default is '1mo')
            period = "max",

            # fetch data by interval (including intraday if period < 60 days
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = "1d",

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            group_by = 'column',

            # adjust all OHLC automatically (open, high, low, close) for dividends & splits
            # (optional, default is False)
            auto_adjust = True,
        
            # Whether to ignore timezone when aligning ticker data from 
            # different timezones. Default is False.
            ignore_tz = True,

            # download pre/post regular market hours data
            # (optional, default is False)
            prepost = False,

            # use threads for mass downloading? (True/False/Integer)
            # (optional, default is True)
            threads = True,

            # proxy URL scheme use use when downloading?
            # (optional, default is None)
            proxy = None
        )
    
else:
    data = pdr.get_data_yahoo(
            # tickers list or string as well
            symbols = tickers,

            # use "period" instead of start/end
            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            # (optional, default is '1mo')
            #period = "max",

            # fetch data by interval (including intraday if period < 60 days
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = 'd',

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            #group_by = 'column',

            # adjust all OHLC automatically (open, high, low, close) for dividends & splits
            # (optional, default is False)
            #auto_adjust = True,

            # download pre/post regular market hours data
            # (optional, default is False)
            #prepost = False,

            # use threads for mass downloading? (True/False/Integer)
            # (optional, default is True)
            #threads = True,

            # proxy URL scheme use use when downloading?
            # (optional, default is None)
            #proxy = None
        )

# remove timezone info
# https://stackoverflow.com/questions/17159207/change-timezone-of-date-time-column-in-pandas-and-add-as-hierarchical-index
# data.index = data.index.tz_convert(None)

#print(data.head())
if write_xlsx:
    rev_data = data['Close'].iloc[::-1]    
    rev_data.iloc[:800].to_excel('vg_shareprices.xlsx', sheet_name='VG Data')

# Derived Data

In [ ]:
ddata = data['Close'].iloc[::-1]  
ddata.head()